# PyCoBi Bifurcation with Auto-07p 

In [ ]:
from pycobi import 

# Initialize a PyCoBi object
pcb = pycobi()

# Use the Hodgkin-Huxley model as your system of ODEs
# Assuming the fortran version of the Hodgkin-Huxley model is available in a file named 'hodgkin_huxley.f'
pcb.load_equations('hodgkin_huxley.f')

# Set the initial conditions and parameters for your system
pcb.set_initial_conditions([V_init, 0.05, 0.6, 0.32])

# Define your control parameter and its range for the continuation
pcb.set_control_parameter('I', start=0, stop=100, step=1)

# Run the continuation
pcb.run_continuation()

# Display the bifurcation diagram
pcb.plot_bifurcation_diagram()